In [2]:
import numpy as np
import h5py
import pandas as pd
from glob import glob
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
# settings
color_pal = sns.color_palette()
plt.style.use("fivethirtyeight")

np.set_printoptions(precision=2)

from IPython.display import clear_output

# grb codes
import PyBlastAfterglowMag as PBA
try:
    import afterglowpy as grb
except:
    afterglowpy = False
    print("Error! could not import afteglowpy")


# Jet Settings (TopHat)


In [83]:
workingdir = os.getcwd()+'/'
pba_src = "/home/vsevolod/Work/GIT/GitHub/PyBlastAfterglowMag/src/pba.out"
loglevel= "info"
structure = {
    "struct":"tophat",
    "Eiso_c":1.e52, "Gamma0c": 300., "M0c": -1.,
    "theta_c": 0.3, "theta_w": 0.3, "nlayers_pw": 80, "nlayers_a": 1
}

In [ ]:
# run code 
def make_runs():

    PBA.parfile_tools.modify_parfile_par_opt(workingdir=workingdir, part="grb",keep_old=False,newparfile="parfile.par",
                                             newpars={},newopts={"method_synchrotron":"Joh06",
                                                                 "method_shock_ele":"analytic",
                                                                 "method_ssc":"none",
                                                                 "method_ne" :"useNe", # useNe
                                                                 "method_eats":"piece-wise",
                                                                 "method_comp_mode": "observFlux",
                                                                 "fname_light_curve":"lc_syn.h5",
                                                                 "fname_spectrum":"spec_syn.h5"})
    pba_id = PBA.id_analytic.JetStruct(n_layers_pw=structure["nlayers_pw"], n_layers_a=structure["nlayers_a"])
    pba_id.save_1d_id(*pba_id.get_1D_id(pars=structure, type="piece-wise"), outfpath=workingdir+"grb_id.h5")
    pba0 = PBA.interface.PyBlastAfterglow(workingdir=workingdir, readparfileforpaths=True, parfile="parfile.par")
    pba0.run(path_to_cpp_executable=pba_src,loglevel=loglevel)
    clear_output(wait=False)
    
    PBA.parfile_tools.modify_parfile_par_opt(workingdir=workingdir, part="grb",keep_old=False,newparfile="parfile.par",
                                             newpars={},newopts={"method_synchrotron":"Dermer09",
                                                                 "method_shock_ele":"analytic",
                                                                 "method_ssc":"none",
                                                                 "method_ne" :"useNe", # useNe
                                                                 "method_eats":"adaptive",
                                                                 "method_comp_mode": "comovSpec",
                                                                 "fname_light_curve":"lc_syn.h5",
                                                                 "fname_spectrum":"spec_syn.h5"})
    pba_id = PBA.id_analytic.JetStruct(n_layers_pw=structure["nlayers_pw"], n_layers_a=structure["nlayers_a"])
    pba_id.save_1d_id(*pba_id.get_1D_id(pars=structure, type="adaptive"), outfpath=workingdir+"grb_id.h5")
    pba = PBA.interface.PyBlastAfterglow(workingdir=workingdir, readparfileforpaths=True, parfile="parfile.par")
    pba.run(path_to_cpp_executable=pba_src,loglevel=loglevel)
    clear_output(wait=False)


    PBA.parfile_tools.modify_parfile_par_opt(workingdir=workingdir, part="grb",keep_old=False,newparfile="parfile.par",
                                             newpars={},newopts={"method_synchrotron":"GSL",
                                                                 "method_shock_ele":"numeric",
                                                                 "method_ssc":"none",
                                                                 "method_ne" :"useNe", # useNe
                                                                 "method_eats":"adaptive",
                                                                 "method_comp_mode": "comovSpec",
                                                                 "fname_light_curve":"lc_ssc.h5",
                                                                 "fname_spectrum":"spec_ssc.h5"})
    pba_id = PBA.id_analytic.JetStruct(n_layers_pw=structure["nlayers_pw"], n_layers_a=structure["nlayers_a"])
    pba_id.save_1d_id(*pba_id.get_1D_id(pars=structure, type="adaptive"), outfpath=workingdir+"grb_id.h5")
    pba_num = PBA.interface.PyBlastAfterglow(workingdir=workingdir, readparfileforpaths=True, parfile="parfile.par")
    pba_num.run(path_to_cpp_executable=pba_src,loglevel=loglevel)
    # clear_output(wait=False)
    
    return (pba0, pba, pba_num)
def plot_runs(pba0,pba1, pba2):
    
    
    
    obs_freq=1e9
    
    print(pba2.GRB.get_lc_totalflux(freq=obs_freq,time=None,spec=False))
    fig, ax = plt.subplots(ncols=1, nrows=1)
    
    ax.plot(pba0.GRB.get_lc_times(spec=False,unique=True),
            pba0.GRB.get_lc_totalflux(freq=obs_freq,time=None,spec=False),
            color="green",label="piece-wise")
    ax.plot(pba1.GRB.get_lc_times(spec=False,unique=True),
            pba1.GRB.get_lc_totalflux(freq=obs_freq,time=None,spec=False),
            color="green",label="Analytic")
    ax.plot(pba2.GRB.get_lc_times(spec=False,unique=True),
            pba2.GRB.get_lc_totalflux(freq=obs_freq,time=None,spec=False),
            color="blue",label="Numeric")
    ax.set_xlabel("time [s]")
    ax.set_ylabel("Flux [mJy]")
    ax.set_xscale("log")
    ax.set_yscale("log")
    # ax.set_ylim(pba1.GRB.get_lc_totalflux(freq=obs_freq,time=None,spec=False).max()*1e-5,
    #             pba1.GRB.get_lc_totalflux(freq=obs_freq,time=None,spec=False).max()*1e2)
    ax.legend()
    plt.title("Radio light curves for Top-Hat jet")
    plt.tight_layout()
    plt.show()
    
plot_runs(*make_runs())

[ INFO    ] : [ main.cpp:152 ] : Computation tgrid = [100, 9.63829e+17] n=1000
[ INFO    ] : [ main.cpp:153 ] : Output      tgrid = [100, 9.63829e+17] n=1000
[ INFO    ] : [ model_magnetar.h:569 ] : Magnetar is not initialized and will not be considered.
[ INFO    ] : [ ejecta_id.h:206 ] :  1D ID has theta_wing=0.3 theta_core=0.3
[ INFO    ] : [ ejecta_id.h:258 ] : Initial data loaded with nshells=1 m_nlayers=1
[ INFO    ] : [ ejecta_id.h:280 ] : Angular grids are initialized. nshells=1 m_nlayers=1
[ INFO    ] : [ ejecta_id.h:303 ] : Energy and mass are rescaled.
[ WARNING ] : [ blastwave_base.h:104 ] : Parameter theta_max not given. Using default value=1.5708
[ WARNING ] : [ blastwave_base.h:107 ] : Parameter epsilon_e_rad not given. Using default value=0
[ WARNING ] : [ blastwave_radiation.h:975 ] : Parameter theta_max not given. Using default value=1.5708
[ WARNING ] : [ blastwave_radiation.h:982 ] : Parameter theta_max not given. Using default value=1.5708
[ WARNING ] : [ eats.h:93